# Librerias
Antes de proceder, necesitamos instalar las librerias necesarias. En nuestro caso, trabajaremos con la Libreria __[scikit-learn](https://scikit-learn.org/stable/index.html)__ en su version __0.24.2__. Esta libreria ofrece una amplia gama de algoritmos y/o tecnicas aplicadas no solo a la inteligencia artificial, sino tambien a otras areas como ciencia de datos. Para poder instalar nuestra libreria en la maquina virtual procederemos a ejecutar el siguiente comando: `!pip install -U scikit-learn==0.24.2`. La ejecucion puede tardar unos momentos.

In [ ]:
# Instalacion de libreria
!pip install -U scikit-learn==0.24.2

# I. Practica-1: tic-tac-toe

In [ ]:
# Importar librerias
import matplotlib.pyplot as plt
import sklearn
import numpy as np
# data sets
from sklearn.datasets import fetch_openml
# division de datos
from sklearn.model_selection import train_test_split
# pre-procesamiento
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler
# red neuronal
from sklearn.neural_network import MLPClassifier

Ejecute la siguiente celda para comprobar que tiene instalada la version de __sklearn 0.24.2__.

In [ ]:
# comprobar version
print('The scikit-learn version is {}.'.format(sklearn.__version__))
# Nota: asegurese de que la version sea: 0.24.2

In [ ]:
# descargar data set
raw = fetch_openml(name = 'tic-tac-toe')

In [ ]:
# extraer datos
dataset = raw['data'].copy()
# extraer target y
dataset['y'] = raw['target'].values

In [ ]:
# 1.1. Imprima el total de filas y columnas
################# Ingrese su codigo en esta celda #####################


In [ ]:
# 1.2. Muestre por medio de codigo el nombre de columnas
################# Ingrese su codigo en esta celda #####################


__Hint:__ El comando `print` es usado para imprimir resultados. Por ejemplo `print(5)` imprime el numero 5. Cuando se crea un __dataset__, existen dos propiedades que se pueden acceder mediante `.`, por ejemplo `dataset.shape` y `dataset.columns`, donde `shape` representa las dimensiones (filas y columnas) y `columns` muestra las columnas.

In [ ]:
# ejecute la celda para visualizar la distribucion de targets
labels, counts = np.unique(dataset['y'].values, return_counts = True)
plt.bar(labels, counts, align = 'center')

### 1. Pregunta: El siguiente plot muestra la distribucion de los targets. Como se observa existe un desbalance de clases.
* Como podria afectar este fenomeno al aprendizaje de la red
* Que enfoque podriamos usar para mitigar el desbalance. Describa en detalle y justifique su funcionamiento.

---

# II. Mapeo
En el ejemplo de clase usamos la siguiente configuracion para mapear los valores en nuestro data set a valores numericos:
* __x__: 1
* __o__: 0
* __b__: -1
* __positive__: 1
* __negative__: 0

Para ello creamos un diccionario llamado __valores_de_mapeo:__
```python
valores_de_mapeo = {'x': 1, 'o': 0, 'b': -1, 'positive': 1, 'negative': 0}
```
En esta seccion definira nuevos valores numericos para cada elemento. Recuerde que los elementos: __x, o, b__ deben de tener valores diferentes entre si. Por ejemplo si asigna 100 a __x__ (`valores_de_mapeo = {'x': 100, ...}`), los valores para __o__ y __b__ deben ser diferentes de 100. Finalmente __asegurese de mapear valores para los targets positive y negative como 1 y 0 respectivamente__. En la siguiente celda cambie el valor de la variable `valores_de_mapeo` reemplazando los `...` por su codigo.

In [ ]:
# 2.1 Defina un diccionario de mapeo de valores
################# Ingrese su codigo en esta celda #####################
valores_de_mapeo = ...

Ejecute las siguientes celdas una vez que haya definido sus valores de mapeo.

In [ ]:
# transformacion
funcion_de_mapeo = {}
for column in dataset.columns:
    funcion_de_mapeo[column] = valores_de_mapeo

In [ ]:
# aplicacion de la funcion de mapeo al data set
dataset.replace(funcion_de_mapeo, inplace = True)

Ejecute la siguiente celda para comprobar sus resultados. Debe ser capaz de visualizar el data set con los nuevos valores asignados.

In [ ]:
dataset.head()

### 2. Pregunta: Que criterio uso para los valores de mapeo?

---

# III. Division de datos
En esta seccion dividiremos el data set en dos grupos: __Entrenamiento__ y __Test__. Usaremos los datos de entrenamiento para alimentar a nuestra red neuronal. Una vez completado el entrenamiento, procederemos a evaluar el modelo usando los datos de __Test__. Para la divicion usaremos las siguientes variables:
* __X_train:__ Datos de entrenamiento.
* __y_train:__ Targets de los datos de entrenamiento.
* __X_test:__ Datos de test.
* __y_test:__ Target de los datos de test.

Recordemos que en clase usamos el comando `train_test_split(...)` con la siguiente configuracion:
```python
X_train, X_test, y_train, y_test = train_test_split(dataset[columnas], dataset['y'], test_size = 0.2, random_state = 1)
```
Para esta ocacion, vamos a agregar un parametro extra: `stratify = dataset['y']`. Este parametro tratara de __balancear__ la division de clases teniendo en cuenta la cantidad de las mismas. La idea es crear una distribucion mas __uniforme__. Reservaremos un __10% para los datos de Test__.

__Nota:__ Asigne  una __semilla__ a `random_state` con valor igual a __1__. Para especificar el porcentaje de los datos de test use el parametro `test_size` especificando valores de porcentajes. En el ejemplo, `test_size = 0.2` representa un __20%__ de datos de test, si se reemplaza por `test_size = 0.15` representaria __15%__. Recuerde reemplazar los `...` en su codigo.

In [ ]:
# seleccionaremos las columnas sin targets
columnas = list(dataset.columns[:-1])
print(columnas)

In [ ]:
################# Ingrese su codigo aqui #####################
# 3.1. Divida los datos en los bloques mencionados.
X_train, X_test, y_train, y_test = ...

Ejecute la siguiente celda para verificar sus resultados. Dede de obtener el resultado: `(862, 9)`

In [ ]:
print(X_train.shape)

---

# IV. Pre-procesamiento
Antes de entrenar nuestra red neuronal, necesitamos pre-procesar los datos. Este es un paso __necesario__. Aplicaremos el pre-procesamiento a los datos de __entrenamiento (X_train)__ y luego a los datos de __test (X_test)__. Para esta seccion vamos a elejir de una lista de __3 tipos de pre-procesamiento__. A continuacion encontrara una lista detallando cada uno de ellos:
* StandardScaler: Aqui las columnas se estandarizan usando: $z = (x - \mu) / s$. Donde x representa los datos, $\mu$ es la media y $s$ la desviacion estandar. Consulte mas detalles de uso [aqui](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler).
* MinMaxScaler: Transforma las columnas al escalarlas a un rango seleccionado. El calculo es dado por: $x = s * (max - min) + min$, donde $s = (x - x_{min} / (x_{max} - x_{min}))$ y __max__ y __min__ son intervalos dados. Consulte mas detalles de uso [aqui](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html#sklearn.preprocessing.MinMaxScaler).
* MaxAbsScaler: Escala cada columna usando su valor absoluto. Consulte mas detalles de uso [aqui](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MaxAbsScaler.html#sklearn.preprocessing.MaxAbsScaler).

Como se vio en clase, para el pre-procesamiento se uso __StandardScaler__ de la siguiente manera:
```python
# crear scalador
scaler = StandardScaler()
# aplicar SOLO A DATOS DE ENTRENAMIENTO
scaler.fit(X_train)
# transformar datos de entrenaminto
X_train = scaler.transform(X_train)
# transformar datos de test
X_test = scaler.transform(X_test)
```
Para poder cambiar por otro pre-procesamiento, cambie la clase: `StandardScaler()` por `MinMaxScaler()` o `MaxAbsScaler()`. Por ejemplo para `MinMaxScaler()`:

```python
scaler = MinMaxScaler()
# aplicar SOLO A DATOS DE ENTRENAMIENTO
scaler.fit(X_train)
# transformar datos de entrenaminto
X_train = scaler.transform(X_train)
# transformar datos de test
X_test = scaler.transform(X_test)
```

Para `MaxAbsScaler()`:

```python
scaler = MaxAbsScaler()
# aplicar SOLO A DATOS DE ENTRENAMIENTO
scaler.fit(X_train)
# transformar datos de entrenaminto
X_train = scaler.transform(X_train)
# transformar datos de test
X_test = scaler.transform(X_test)
```
__Nota:__ Recuerde asignar el pre-procesamiento a la variable __scaler__, ej: `scaler = StandardScaler()`, `scaler = MinMaxScaler()` o `scaler = MaxAbsScaler()` respectivamente. En la siguiente asigne un pre-procesamiento a la variable `scaler` (remueva los `...`)

In [ ]:
################# Ingrese su codigo aqui #####################
# 4.1. Asigne un pre-procesamiento
scaler = ...
# aplicar SOLO A DATOS DE ENTRENAMIENTO
...
# transformar datos de entrenaminto
X_train = ...
# transformar datos de test
X_test = ...

### 3. Pregunta: Jusfifique la eleccion del tipo de pre-procesamiento usado.

---

# V. Creacion de Red Neuronal

<div align="center">
<img src="https://raw.githubusercontent.com/aguilarls/practicas/main/Redes%20Neuronales/images/red-03.png" />
</div>

Usualmente, cuando se define una red se habla de __arquitectura__. En ese sentido, la arquitectura de la red representa componentes de la misma. En esta seccion implementara la red neuronal mostrada en el diagrama. Para ello debera crear las capas ocultas. Como se observa existen __dos capas ocultas__ compuestas por __50 neuronas__ cada una. Para la creacion de la red usaremos la clase: [MLPClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn-neural-network-mlpclassifier). Para poder especificar el numero de capas ocultas y neuronas usaremos una __tupla__. En programacion la __tupla__ es una estructura usada para representar informacion. En python se definen mediante `()`:
```python
capas_ocultas = (15, 10)
```
En el ejemplo, estamos definiendo una tupla, que contiene 2 elementos. El primero es el valor __15__, y el segundo __10__. De este ejemplo se deduce que en la primera capa oculta existen __15 unidades__ y __10 en la segunda__. Para el caso de la red, especificaremos las capas ocultas con el comando: `hidden_layer_sizes = capas_ocultas`. Asimismo usaremos el comando `max_iter` para definir el __numero de iteraciones__. Finalmente, definiremos el comportamiento aleatorio con `random_state = 1`. En el siguiente bloque tendra que asignar los valores descritos.

En la siguiente celda defina las capas ocultas en la variable `capas_ocultas`. Asimismo asigne un numero de iteraciones en la variable `iteraciones`. Recuerde remover los `...`

In [ ]:
################# Ingrese su codigo aqui #####################
# 5.1. Defina la arquitectura de su red neuronal
capas_ocultas = ...
iteraciones = ...

In [ ]:
# creacion de la red
red_neuronal = MLPClassifier(random_state = 1, max_iter = iteraciones, hidden_layer_sizes = capas_ocultas)

In [ ]:
################# Ingrese su codigo aqui #####################
# 5.2. Inspeccione los parametros de la red


__hint:__ El comando: `red_neuronal.get_params()` muestra la lista de atributos de su red. El ratio de aprendizaje se suele tambien llamar __learning_rate_init__, y el optimizador __solver__.

### 4. Pregunta: Cuales son el ratio de aprendizaje y optimizador usados por su red?

---

# VI. Entrenamiento
En esta seccion entrenaremos la red neuronal con los los datos pre-procesados. Para entrenar la red, necesitamos llamar al comando:
```python
red_neuronal.fit(X_train, y_train)
```
Donde __X_train__ e __y_train__ representan los datos de entrenamiento y/o targets. De manera general, la funcion `fit(X, y)` espera que el primer argumento sean los datos y el segundo los targets. En la siguiente celda implemente el entrenamiento de la red.

In [37]:
################# Ingrese su codigo aqui #####################
# 6.1. Entrene la red 


---

# VII. Predicciones
Una vez completada la fase de entrenamiento, vamos a realizar algunas predicciones. Para ello, solo necesitaremos proporcionar datos al modelo. Para este ejemplo vamos a usar la primera fila de los datos de entrenamiento: `X_train_scaler[:1]`. Ahora, en este caso, disponemos de dos formas de visualizar las predicciones: probabilidades y targets. Si usamos probabilidades tendriamos:
```python
red_neuronal.predict_proba(X_train[:1])
```
Mientras que si usamos los targets el resultado seria:
```python
red_neuronal.predict(X_test[:1])
```
Podemos verificar los resultados observando los primeros valores de los targets de entrenamiento y test con `y_train[0]` e `y_test[0]`
```python
print(y_train.values[0])
```

```python
print(y_test.values[0])
```

In [ ]:
# Probabilidades de la primera fila (datos de entrenamiento)
print(red_neuronal.predict_proba(X_train[:1]))
# Valor de target de la primera fila (datos de entrenamiento)
print(red_neuronal.predict(X_train[:1]))
# Verificar valor real del target (datos de entrenamiento) 
print(y_train.values[0])

In [ ]:
# Probabilidades de la primera fila (datos de test)
print(red_neuronal.predict_proba(X_test[:1]))
# Valor de target de la primera fila (datos de entrenamiento)
print(red_neuronal.predict(X_test[:1]))
# Verificar valor real del target (datos de entrenamiento) 
print(y_test.values[0])

### 5. Pregunta: Coincidieron las predicciones para los datos de entrenamiento y test?. Describa una hipotesis acerca del comportamiento observado.

---

# VIII. Evaluacion
Una vez entrenado el modelo, podemos realizar predicciones, no obstante, necesitamos __evaluar__ la __presicion (accuracy)__ de las predicciones. Para ello usaremos los datos de entrenamiento y test conjuntamente con los targets. Para ello usaremos el comando `score`. Para los datos de entrenamiento:
```python
red_neuronal.score(X_train, y_train)
```
Ahora para test:
```python
red_neuronal.score(X_test, y_test)
```

In [ ]:
# Accuracy para entrenamiento
print(red_neuronal.score(X_train, y_train))

In [ ]:
################# Ingrese su codigo aqui #####################
# 8.1. Imprima el accuracy para test


### 6. Pregunta: Cuales son los porcentajes de accuracy para los datos de entrenamiento y test?. Explique el comportamiento observado elaborando una hipotesis.

# IX. Concluciones:
* En este Notebook ha aprendido cual es el proceso durante el entrenamiento de una red Neuronal.
* Si bien es cierto uso la libreria __scikit-learn__, esta no es la unica alternativa disponible. De hecho existen diversas herramientas (librerias) con las que se puede trabajar. No obstante le recordamos siempre considerar aquellas de __codigo abierto__ (cuando la situacion lo requiera), ya que muchos modelos complejos se entrenan usando __software libre__.
* Entrenar redes es una tarea ardua que involucra muchos factores, como la arquitectura de la red. De hecho diferentes arquitecturas resultan en performances variadas.
* Es importante recordar que los datos de entrenamiento se usan para la red. Los datos de test se usan para evaluar. Es importante que no mezcle los datos, ya que obtendra resultados __sobre estimados__.
* Existen diversas formas de division de datos. En este ejemplo hemos usado un 90-10%, donde 90% para entrenamiento y 10% para test. No obstante alternativas como la validacion cruzada y otras tambien se pueden aplicar.


### 7. Pregunta: Despues de haber completado el Notebook, elabore un resumen resaltando los puntos mas importantes que ha encontrado.

---